In [1]:
cd ..

/Users/ali/Documents/tomfoolery/oleorcarl


In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV 
from sklearn.utils.fixes import loguniform
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC



In [22]:
from src.database import Student
import numpy as np

students = list(Student.select())

X = np.stack([student.face for student in students if student.face is not None])

y = np.array([
    bool(student.school == "stolaf")
    for student in students
    if student.face is not None
])  # fmt: skip

print(X.shape, y.shape)


(862, 128) (862,)


In [50]:
# use grid search to find the best values for C and gamma

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.svm import SVC


# ranges from https://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf (p. 6)

param_grid = [
    {
        "svc__kernel": ["linear"],
        "svc__C": np.logspace(-3, 6, 10, base=2),
    },
    {
        "svc__kernel": ["rbf"],
        "svc__C": np.logspace(-5, 15, 5, base=2),
        "svc__gamma": np.logspace(-15, 3, 5, base=2),
    },
    {
        "svc__kernel": ["poly"],
        "svc__C": np.logspace(-5, 15, 5, base=2),
        "svc__degree": [2, 3, 4],
        "svc__gamma": np.logspace(-15, 3, 5, base=2),
    },
]

model = GridSearchCV(
    estimator=make_pipeline(StandardScaler(), SVC()),
    param_grid=param_grid,
    scoring="accuracy",
    n_jobs=12,  # use all available threads
    refit=True,  # when done, fit the model with the best parameters found
    cv=5,  # 5-fold CV
)

model.fit(X, y)

print(model.best_params_, model.best_score_, model.refit_time_)

model


{'svc__C': 4.0, 'svc__kernel': 'linear'} 0.6670453017878748 1.4138727188110352


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svc', SVC())]),
             n_jobs=12,
             param_grid=[{'svc__C': array([ 0.125,  0.25 ,  0.5  ,  1.   ,  2.   ,  4.   ,  8.   , 16.   ,
       32.   , 64.   ]),
                          'svc__kernel': ['linear']},
                         {'svc__C': array([3.1250e-02, 1.0000e+00, 3.2000e+01, 1.0240e+03, 3.2768e+04]),
                          'svc__gamma': array([3.05175781e-05, 6.90533966e-04, 1.56250000e-02, 3.53553391e-01,
       8.00000000e+00]),
                          'svc__kernel': ['rbf']},
                         {'svc__C': array([3.1250e-02, 1.0000e+00, 3.2000e+01, 1.0240e+03, 3.2768e+04]),
                          'svc__degree': [2, 3, 4],
                          'svc__gamma': array([3.05175781e-05, 6.90533966e-04, 1.56250000e-02, 3.53553391e-01,
       8.00000000e+00]),
                          'svc__kernel': ['poly']}],
             scoring='accuracy')

In [ ]:
from sklearn.model_selection import LeaveOneOut

# (hopefully do this in a thread pool of some sort)

# for every student in the database
    # train a classifier on everyone else
    # store the resulting score for that student